In [13]:
from sympy import *

In [14]:
t=symbols('t')
c1=Symbol('c1',positive=True)
c2=Symbol('c2',positive=True)
k=Symbol('k',positive=True)
A=Function('A')
N=Function('N')

In [15]:
eq1=Equality(diff(A(t)),-c1*A(t))
eq2=Equality(diff(N(t)), k*A(t)-c2*N(t))
display(eq1)
display(eq2)

Eq(Derivative(A(t), t), -c1*A(t))

Eq(Derivative(N(t), t), -c2*N(t) + k*A(t))

In [16]:
A0,N0=symbols('A0 N0') # initial concentrations
Asoln=dsolve(eq1,A(t),ics={A(0):A0})
Asoln

Eq(A(t), A0*exp(-c1*t))

In [17]:
Nsoln=dsolve(eq2.subs(A(t),Asoln.rhs),N(t),ics={N(0):N0})
#Nsoln.simplify()
Nsoln

Eq(N(t), (A0*k*Piecewise((-exp(c2*t)/(c1*exp(c1*t) - c2*exp(c1*t)), Ne(c1, c2)), (t, True)) + Piecewise(((A0*k + N0*(c1 - c2))/(c1 - c2), Ne(c1, c2)), (N0, True)))*exp(-c2*t))

In [18]:
Nsoln.simplify() 

Eq(N(t), Piecewise(((-A0*k + (A0*k + N0*(c1 - c2))*exp(t*(c1 - c2)))*exp(-c1*t)/(c1 - c2), Ne(c1, c2)), ((A0*k*t + N0)*exp(-c2*t), True)))

With Zeroth Order source
==


In [19]:
# Additionally need s_a, a zeroth order NH4 source, and s_n
# a zeroth order NO3 source.
src_a,src_n = symbols('s_a s_n')
eq1=Equality(diff(A(t)),-c1*A(t) + src_a)
eq2=Equality(diff(N(t)), k*A(t)-c2*N(t) + src_n).subs(src_n,0)
display(eq1)
display(eq2)

Eq(Derivative(A(t), t), -c1*A(t) + s_a)

Eq(Derivative(N(t), t), -c2*N(t) + k*A(t))

In [10]:
A0,N0=symbols('A0 N0') # initial concentrations
Asoln=dsolve(eq1,A(t),ics={A(0):A0})
Asoln

Eq(A(t), (s_a + exp(c1*(-t + log(A0*c1 - s_a)/c1)))/c1)

In [20]:
Asoln.simplify().expand()

Eq(A(t), A0*exp(-c1*t) + s_a/c1 - s_a*exp(-c1*t)/c1)

Wolfram Alpha, given
```
  da/dt = -s1 * a + s2
```  
returns
$$a(t) = C_1 e^{-s1 t} + s2/s1$$
or substituting my variables,
$$A(t) = C_1 e^{-c_1 t} + \frac{s_a}{c_1} $$

In [24]:
# So I could rewrite this in the slightly nicer form:
Asoln=Equality(A(t),(A0-src_a/c1)*E**(-c1*t) + src_a/c1)
Asoln

Eq(A(t), (A0 - s_a/c1)*exp(-c1*t) + s_a/c1)

In [28]:
Neq=eq2.subs(A(t),Asoln.rhs)
Neq

Eq(Derivative(N(t), t), -c2*N(t) + k*((A0 - s_a/c1)*exp(-c1*t) + s_a/c1))

In [25]:
Nsoln=dsolve(eq2.subs(A(t),Asoln.rhs.simplify()),N(t),ics={N(0):N0})
Nsoln

Eq(N(t), (Piecewise(((N0*c2*(c1 - c2) + k*(A0*c2 - s_a))/(c2*(c1 - c2)), Ne(c1, c2)), (N0 - k*s_a/(c1*c2), True)) + Piecewise((-A0*c1*c2*k*exp(c2*t)/(c1*c2*exp(c1*t) - c2**2*exp(c1*t)) + c1*k*s_a*exp(c1*t)*exp(c2*t)/(c1*c2*exp(c1*t) - c2**2*exp(c1*t)) - c2*k*s_a*exp(c1*t)*exp(c2*t)/(c1*c2*exp(c1*t) - c2**2*exp(c1*t)) + c2*k*s_a*exp(c2*t)/(c1*c2*exp(c1*t) - c2**2*exp(c1*t)), Ne(c1, c2)), (A0*c2*k*t - k*s_a*t + k*s_a*exp(c2*t)/c2, True))/c1)*exp(-c2*t))

In [26]:
soln=Nsoln.simplify()
soln

Eq(N(t), Piecewise(((c1*(N0*c2*(c1 - c2) + k*(A0*c2 - s_a))*exp(t*(c1 - c2)) - k*(A0*c1*c2 - c1*s_a*exp(c1*t) + c2*s_a*exp(c1*t) - c2*s_a))*exp(-c1*t)/(c1*c2*(c1 - c2)), Ne(c1, c2)), ((N0*c1*c2 - k*s_a + k*(c2*t*(A0*c2 - s_a) + s_a*exp(c2*t)))*exp(-c2*t)/(c1*c2), True)))

In [49]:
Nsoln2=soln.rhs.args[0].args[0]
Nsoln2

(c1*(N0*c2*(c1 - c2) + k*(A0*c2 - s_a))*exp(t*(c1 - c2)) - k*(A0*c1*c2 - c1*s_a*exp(c1*t) + c2*s_a*exp(c1*t) - c2*s_a))*exp(-c1*t)/(c1*c2*(c1 - c2))

Wolfram Alpha with an input of:
```
dn/dt = -s1 * n + s2 * exp(-s3*t) + s4
```
Returns 
$$ N(t) = C_1 e^{-s1 t} + \frac{s2 e^{-s3 t}}{s1-s3} + \frac{s4}{s1}$$


In [21]:
C_1,s1,s2,s3,s4= symbols('C_1 s1 s2 s3 s4')
NsolnWA=C_1 * E**(-s1 * t) + (s2 * E**(-s3*t))/(s1-s3) + s4/s1
NsolnWA

C_1*exp(-s1*t) + s2*exp(-s3*t)/(s1 - s3) + s4/s1

In [53]:
# And sub back in what all those things really area:
NsolnWA2=NsolnWA.subs(s1,c2).subs(s2,k*(A0-src_a/c1)).subs(s3,c1).subs(s4,k*src_a/c1)
C_1_value=solve(NsolnWA2.subs(t,0)-N0,C_1)[0]
NsolnWA3=NsolnWA2.subs(C_1,C_1_value)
Equality(N(t),NsolnWA3)

Eq(N(t), k*(A0 - s_a/c1)*exp(-c1*t)/(-c1 + c2) + (A0*c2*k + N0*c1*c2 - N0*c2**2 - k*s_a)*exp(-c2*t)/(c2*(c1 - c2)) + k*s_a/(c1*c2))

In [52]:
# verify it's the same.
(NsolnWA3 - Nsoln2).expand().simplify()

0

In [22]:
# The ammonium zeroth order source already generates a zeroth order
# nitrate source term, s4
# 
# And sub back in what all those things really area:
NsolnWA4=NsolnWA.subs(s1,c2).subs(s2,k*(A0-src_a/c1)).subs(s3,c1).subs(s4,k*src_a/c1+src_n)
C_1_value=solve(NsolnWA4.subs(t,0)-N0,C_1)[0]
NsolnWA5=NsolnWA4.subs(C_1,C_1_value)
Equality(N(t),NsolnWA5)

Eq(N(t), k*(A0 - s_a/c1)*exp(-c1*t)/(-c1 + c2) + (s_n + k*s_a/c1)/c2 + (A0*c2*k + N0*c1*c2 - N0*c2**2 - c1*s_n + c2*s_n - k*s_a)*exp(-c2*t)/(c2*(c1 - c2)))